### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


### Drop Null target records

In [3]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [4]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [5]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [6]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [9]:
features_list = [
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
    
#ID Features
'aggs907',
'dm216s',
'dm206s',
'balmag01',
'g406s',

# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn',             
'month_year',
'60_in_15mob']

In [10]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [11]:
oot_df = oot_df1[features_list]

In [12]:
model_df.shape, oot_df.shape

((101984, 40), (17343, 40))

In [13]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [14]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [15]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [16]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [17]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3031153/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [18]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3031153/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [19]:
X_oot.shape,y_oot.shape

((17343, 38), (17343,))

### Split Model dataset into Train and Test

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [21]:
X_train.shape, X_test.shape

((61190, 38), (40794, 38))

In [22]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 38) (12238, 38) (40794, 38)


In [23]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [24]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 7,
    'alpha': 4.6,
    'gamma': 5.2,
   
    'scale_pos_weight': 4,
    'random_state': 42
    
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 1000,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.63393	eval-auc:0.61686
[10]	train-auc:0.67000	eval-auc:0.65209
[20]	train-auc:0.67266	eval-auc:0.65492
[30]	train-auc:0.67453	eval-auc:0.65741
[40]	train-auc:0.67750	eval-auc:0.65917
[50]	train-auc:0.67806	eval-auc:0.66018
[60]	train-auc:0.67891	eval-auc:0.66036
[70]	train-auc:0.67926	eval-auc:0.66045
[80]	train-auc:0.68086	eval-auc:0.66182
[90]	train-auc:0.68224	eval-auc:0.66300
[100]	train-auc:0.68273	eval-auc:0.66385
[110]	train-auc:0.68351	eval-auc:0.66465
[120]	train-auc:0.68436	eval-auc:0.66543
[130]	train-auc:0.68484	eval-auc:0.66581
[140]	train-auc:0.68534	eval-auc:0.66601
[150]	train-auc:0.68647	eval-auc:0.66682
[160]	train-auc:0.68694	eval-auc:0.66700
[170]	train-auc:0.68767	eval-auc:0.66721
[180]	train-auc:0.68845	eval-auc:0.66777
[190]	train-auc:0.68933	eval-auc:0.66834
[200]	train-auc:0.68999	eval-auc:0.66882
[210]	train-auc:0.69034	eval-auc:0.66900
[220]	train-auc:0.69100	eval-auc:0.66955
[230]	train-auc:0.69132	eval-auc:0.67011
[240]	train-auc:0.69197	eva

In [25]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.72, Gini: 0.44, KS: 0.32
Eval AUC: 0.69, Gini: 0.38, KS: 0.29
Test AUC: 0.70, Gini: 0.40, KS: 0.29
OOT AUC: 0.71, Gini: 0.42, KS: 0.30


In [26]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.72, Gini: 0.4395, KS: 0.3215
Test AUC: 0.70, Gini: 0.3970, KS: 0.2868
OOT AUC: 0.71, Gini: 0.4193, KS: 0.3039


In [27]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
35                                         days_on_pg       273.0
25                                              g406s       208.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       184.0
30                           No of enq in past 90days       150.0
19  deviation_from_median_income/standard_deviatio...       128.0
24                                           balmag01       116.0
26                         percentage_ever_delinquent       112.0
11                         bounced_transactions_count       108.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...       102.0
31                          Num_Enq_non_BANK_last_12M       101.0
6                highest_sanction_amount_credit_cards        97.0
12                           Min_Balance_Min_M1_M2_M3        93.0
8             Utilization_CREDIT CARD_opened_last_36M        90.0
1                   num_CONSUMER LOAN_enq_last_6M_36M        89.0
34  MEAN(r

In [28]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.300245         0.582104        0.850510        0.489883   4896   
9          0.186925         0.442913        0.489882        0.404512   4895   
8          0.140960         0.374872        0.404496        0.348015   4895   
7          0.110112         0.326595        0.348009        0.306937   4895   
6          0.094178         0.290131        0.306933        0.274010   4895   
5          0.072523         0.259287        0.273992        0.244410   4895   
4          0.063534         0.230018        0.244399        0.215505   4895   
3          0.048212         0.200754        0.215499        0.185498   4895   
2          0.039019         0.170147        0.185498        0.154293   4895   
1          0.023284         0.132176        0.154292        0.073489   4896   

        label_good  label_bad  
Decile                         
10            3426       1470  
9             3980        915  
8             4205        690  
7             4356        539  
6             4434        461  
5             4540        355  
4             4584        311  
3             4659        236  
2             4704        191  
1             4782        114

In [29]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.287255         0.581133        0.877959        0.488383   4080   
9          0.177494         0.442126        0.488361        0.403544   4079   
8          0.143172         0.374297        0.403526        0.347927   4079   
7          0.113480         0.326953        0.347893        0.307647   4080   
6          0.100515         0.290428        0.307643        0.274246   4079   
5          0.084580         0.259307        0.274233        0.244985   4079   
4          0.066176         0.230258        0.244984        0.215873   4080   
3          0.058838         0.200782        0.215871        0.185667   4079   
2          0.043393         0.170422        0.185662        0.154676   4079   
1          0.031373         0.132736        0.154667        0.069400   4080   

        label_good  label_bad  
Decile                         
10            2908       1172  
9             3355        724  
8             3495        584  
7             3617        463  
6             3669        410  
5             3734        345  
4             3810        270  
3             3839        240  
2             3902        177  
1             3952        128

In [30]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.288184         0.583134        0.843377        0.492618   1735   
9          0.182814         0.443416        0.492580        0.402468   1734   
8          0.133795         0.372639        0.402466        0.344966   1734   
7          0.103806         0.323078        0.344911        0.303128   1734   
6          0.095156         0.286454        0.303127        0.271141   1734   
5          0.081268         0.256173        0.271128        0.241361   1735   
4          0.062860         0.226560        0.241315        0.211497   1734   
3          0.045559         0.197117        0.211486        0.182748   1734   
2          0.038062         0.168435        0.182738        0.153209   1734   
1          0.028242         0.131972        0.153208        0.074086   1735   

        label_good  label_bad  
Decile                         
10            1235        500  
9             1417        317  
8             1502        232  
7             1554        180  
6             1569        165  
5             1594        141  
4             1625        109  
3             1655         79  
2             1668         66  
1             1686         49

In [31]:
#Save the model
xgb_model.save_model('LeoPayu_BBILAP_60_in_15mob.json')
print('Model Saved')

Model Saved


In [32]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILAP_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_BBILAP_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_BBILAP_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_BBILAP_Oot_Pred.csv',index=False)
